In [3]:
import re
import numpy as np
import pandas as pd 
from konlpy.tag import Okt
import pyLDAvis
import pyLDAvis.gensim
from gensim import corpora 
from gensim import models
import gensim

In [2]:
def trans(rowdata,kingname):
    rmhdata = []
    for num1 in range(len(rowdata)):
        for num in range(len(rowdata[num1])):
            rmhdata.append(re.sub('[(一-龥)]','',rowdata[num1][num]))
    wdata =[]
    for i in range(len(rmhdata)):
        wdata.append(re.sub('-|○|/|。|=|\n','',rmhdata[i]))
    
    sdata =[]
    for i in range(len(wdata)):
        if wdata[i] != '':
            sdata.append(wdata[i])
    ssdata =[]
    for i in range(len(sdata)):
        if re.search('[가-힣]+',sdata[i]):
            ssdata.append(sdata[i])
    data =[]
    for i in range(len(ssdata)):
        if kingname not in ssdata[i]:
           data.append(ssdata[i])

    return data

In [4]:
data =[]
for i in range(0,1):
    txt1 = open('단종실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '단종실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '백성' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'단종실록'))

In [5]:
# 시리즈 만들기
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("단종실록"," "))
df=df.map(lambda x : x.replace("단종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))

In [6]:
len(df)

179

In [7]:
okt = Okt()

In [8]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [9]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

In [10]:
# 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary

In [11]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	백성	0.014898005872964859
	하니	0.008890648372471333
	모두	0.007473046891391277
	경태	0.0072917877696454525
	수령	0.00728583475574851
	가지	0.006452878005802631
	청컨대	0.006451538763940334
	다음	0.0051299226470291615
	자가	0.004970726557075977
	만약	0.004952108021825552


Topic ID: 1
	하니	0.012195589952170849
	세조	0.010799380950629711
	백성	0.010460654273629189
	모두	0.008632194250822067
	경태	0.006934737786650658
	군사	0.0058544655330479145
	관찰사	0.005517483688890934
	대군	0.005349420476704836
	우리	0.004960228223353624
	의논	0.004841463174670935


Topic ID: 2
	백성	0.013809598051011562
	모두	0.01374633889645338
	하니	0.007761615328490734
	경태	0.006122487597167492
	수령	0.0060587418265640736
	의논	0.006029598414897919
	고을	0.0060047367587685585
	어찌	0.0050075361505150795
	대신	0.004267601761966944
	장차	0.004260850138962269


Topic ID: 3
	하니	0.01180529035627842
	우리	0.0067513687536120415
	생각	0.006183641497045755
	백성	0.006071524694561958
	비록	0.00573889072984457
	모두	0.005643078126013279
	가지	0.00543467886

In [12]:
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18     0.043007 -0.000725       1        1  18.617134
14    -0.029208 -0.025676       2        1   9.029428
1      0.034026  0.002088       3        1   8.628440
3      0.032876 -0.002320       4        1   7.367234
4      0.016899  0.008027       5        1   7.290674
10    -0.012397 -0.015531       6        1   5.988628
12     0.033177 -0.010027       7        1   5.783476
2      0.005205  0.003090       8        1   5.755644
9      0.030215  0.025350       9        1   5.318030
5      0.022007 -0.028341      10        1   4.631087
16     0.020962 -0.023390      11        1   4.153247
11    -0.030537 -0.028865      12        1   3.476703
7     -0.023849  0.084872      13        1   3.351502
0     -0.018445 -0.003229      14        1   3.236270
8     -0.050043 -0.002760      15        1   2.636300
17     0.014725  0.032186      16        1   1.317090
15     0.004379 -0.008304      17        1   1.064098
13    -0.060477  0.003288      18        1   0.881116
6      0.006081 -0.001045      19        1   0.867900
19    -0.038604 -0.008688      20        1   0.606002, topic_info=     Term        Freq       Total Category  logprob  loglift
29     백성  259.000000  259.000000  Default  30.0000  30.0000
1115   서고   62.000000   62.000000  Default  29.0000  29.0000
332    다음   83.000000   83.000000  Default  28.0000  28.0000
22     모두  281.000000  281.000000  Default  27.0000  27.0000
87     하니  265.000000  265.000000  Default  26.0000  26.0000
...   ...         ...         ...      ...      ...      ...
84    청컨대    0.520410  114.180687  Topic20  -5.8722  -0.2849
192    마음    0.494981  104.525749  Topic20  -5.9223  -0.2466
49     어찌    0.526879  142.283936  Topic20  -5.8599  -0.4926
781    무릇    0.410401   58.872944  Topic20  -6.1097   0.1400
54     의논    0.441140  124.179054  Topic20  -6.0375  -0.5341

[1431 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
6394      2  0.515271    가감
6300      9  0.636057    가격
6395      2  0.513878    가경
817       7  0.610837   가긍하
5188      1  0.628226  가시나무
...     ...       ...   ...
587      11  0.049385    흉년
587      13  0.197541    흉년
587      14  0.049385    흉년
587      15  0.049385    흉년
3458      8  0.637539    흠격

[3373 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 15, 2, 4, 5, 11, 13, 3, 10, 6, 17, 12, 8, 1, 9, 18, 16, 14, 7, 20])

In [13]:
# 주제 : 폐단

In [14]:
data =[]
for i in range(0,1):
    txt1 = open('단종실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '단종실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '폐단' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'단종실록'))

In [15]:
len(df)

179

In [16]:
# 시리즈 만들기
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("단종실록"," "))
df=df.map(lambda x : x.replace("단종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))

In [17]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [18]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]
# 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary

In [19]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	지하	0.009386503137648106
	의논	0.008589385077357292
	대신	0.008284656330943108
	표기	0.006995301228016615
	명령	0.006924530025571585
	정부	0.006166012957692146
	대장	0.005972160492092371
	역사	0.005772479344159365
	각각	0.0053183636628091335
	모두	0.005112122278660536


Topic ID: 1
	백성	0.01593138463795185
	가지	0.010198274627327919
	모두	0.007986197248101234
	의논	0.007073187734931707
	책방	0.0060477470979094505
	생각	0.0059897624887526035
	수령	0.005981095135211945
	수가	0.005966986529529095
	하니	0.005746408831328154
	역사	0.005723041482269764


Topic ID: 2
	대군	0.015592175535857677
	모두	0.008414728567004204
	하니	0.008340495638549328
	어찌	0.00820672046393156
	폐단	0.007996640168130398
	의논	0.007547012995928526
	경태	0.006399297155439854
	더욱	0.006021580193191767
	대신	0.005851163994520903
	온천	0.005133331287652254


Topic ID: 3
	정씨	0.02227315492928028
	노비	0.01438599918037653
	남편	0.013072607107460499
	강순덕	0.01281427126377821
	의논	0.01138551440089941
	이숙번	0.011170757934451103
	모두	0.00934868492186069

In [20]:
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.069694  0.026052       1        1  19.224716
16    -0.023862 -0.063898       2        1  15.048703
13     0.021795 -0.006380       3        1  11.982782
3      0.051841  0.076593       4        1   8.092318
15     0.048869  0.012282       5        1   6.022143
4      0.008928 -0.095755       6        1   6.003664
1      0.041336  0.020601       7        1   4.905403
18     0.018378 -0.000039       8        1   4.683023
6      0.031613 -0.001128       9        1   4.336585
9     -0.004325 -0.021062      10        1   3.923664
12    -0.219163  0.040149      11        1   3.326777
2      0.030499 -0.016778      12        1   3.076416
14     0.036713  0.044439      13        1   2.549778
8     -0.001189  0.002454      14        1   2.509425
10    -0.008538  0.014293      15        1   1.255214
7     -0.041894 -0.018002      16        1   1.021167
19     0.001675  0.012842      17        1   0.878778
17    -0.062141  0.013014      18        1   0.436947
0     -0.000262 -0.050930      19        1   0.425320
5      0.000033  0.011254      20        1   0.297169, topic_info=     Term        Freq       Total Category  logprob  loglift
297    각각   51.000000   51.000000  Default  30.0000  30.0000
25     모두  118.000000  118.000000  Default  29.0000  29.0000
148    고을   38.000000   38.000000  Default  28.0000  28.0000
330    도회   26.000000   26.000000  Default  27.0000  27.0000
2544   정씨   62.000000   62.000000  Default  26.0000  26.0000
...   ...         ...         ...      ...      ...      ...
104    만약    0.123832   56.685780  Topic20  -5.8624  -0.3077
506    제수    0.112802   24.484758  Topic20  -5.9557   0.4385
224    소서    0.113083   25.593786  Topic20  -5.9532   0.3966
37     비록    0.117770   81.513344  Topic20  -5.9126  -0.7212
101    때문    0.116782   67.417633  Topic20  -5.9210  -0.5397

[1359 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1154      3  0.255658   가게
1154     10  0.255658   가게
2435      1  0.290279   가사
2435      4  0.290279   가사
1328      1  0.184684   가산
...     ...       ...  ...
88       14  0.513394  훈도할
468       2  0.683722   훈련
468       4  0.113954   훈련
2594      2  0.384637   희천
2594     14  0.384637   희천

[2284 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 17, 14, 4, 16, 5, 2, 19, 7, 10, 13, 3, 15, 9, 11, 8, 20, 18, 1, 6])